In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################


# HDA Flowsheet Simulation and Optimization
Maintainer: Brandon Paul  
Author: Brandon Paul  
Updated: 2023-06-01  


## Note

This tutorial will be similar to the HDA flowsheet tutorial in the Tutorials section, except that we use a distillation column instead of a second flash (F102) to produce benzene and toluene products.


## Learning outcomes


- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a  flowsheet using Arcs
- Using the SequentialDecomposition tool to initialize a flowsheet with recycle
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

The flowsheet that we will be using for this module is shown below with the stream conditions. We will be processing toluene and hydrogen to produce at least 370 TPY of benzene. As shown in the flowsheet, we use a flash tank, F101, to separate out the non-condensibles, and a distillation column, D101, to further separate the benzene-toluene mixture to improve the benzene purity.  The non-condensibles separated out in F101 will be partially recycled back to M101 and the rest will be purged. We will assume ideal gas behavior for this flowsheet. The properties required for this module are defined in

- `hda_ideal_VLE.py`
- `idaes.models.properties.activity_coeff_models.BTX_activity_coeff_VLE`
- `hda_reaction.py`

We will be using two thermodynamic packages: one (first in the list above) containing all four components (i.e., toluene, hydrogen, benzene, and methane) and the other (second in the list above) containing benzene and toluene only. The latter is needed to simplify the VLE calculations in the distillation column model. 

![](HDA_flowsheet_distillation.png)



## Translator block

Benzene and toluene are separated by distillation, so the process involves phase equilibrium and two-phase flow conditions. However, the presence of hydrogen and methane complicates the calculations. This is because, hydrogen and methane are non-condensable under all conditions of interest; ergo, a vapor phase will always be present, and the mixture bubble point is extremely low. To simplify the phase equilibrium calculations, hydrogen and methane will be considered completely as non-condensable and insoluble in the liquid outlet from the flash F101.

Since no hydrogen and methane will be present in the unit operations following the flash, a different component list can be used to simplify the property calculations. IDAES supports the definition of multiple property packages within a single flowsheet via `Translator` blocks. `Translator` blocks convert between different property calculations, component lists, and equations of state. 

## Importing required pyomo and idaes components


To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the following components from Pyomo:
- Constraint (to write constraints)
- Var (to declare variables)
- ConcreteModel (to create the concrete model object)
- Expression (to evaluate values as a function of variables defined in the model)
- Objective (to define an objective function for optimization)
- SolverFactory (to solve the problem)
- TransformationFactory (to apply certain transformations)
- Arc (to connect two unit models)
- SequentialDecomposition (to initialize the flowsheet in a sequential mode)

For further details on these components, please refer to the pyomo documentation: https://pyomo.readthedocs.io/en/stable/


In [2]:
from pyomo.environ import (
    Constraint,
    Var,
    ConcreteModel,
    Expression,
    Objective,
    TransformationFactory,
    value,
)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Import `Arc` and `SequentialDecomposition` tools from `pyomo.network`
</div>

In [3]:
# Todo: Import the above mentioned tools from pyomo.network
from pyomo.network import Arc, SequentialDecomposition

From IDAES, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- Heater
- CSTR
- Flash
- Separator (splitter) 
- PressureChanger
- Translator (to switch from one property package to another)
- TrayColumn (distillation column)
- CondenserType (Type of the overhead condenser: complete or partial)
- TemperatureSpec (Temperature specification inside the condenser)

In [4]:
from idaes.core import FlowsheetBlock

In [5]:
from idaes.models.unit_models import (
    PressureChanger,
    Mixer,
    Separator as Splitter,
    Heater,
    CSTR,
    Flash,
    Translator,
)

from idaes.models_extra.column_models import TrayColumn
from idaes.models_extra.column_models.condenser import CondenserType, TemperatureSpec

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom. 

In [6]:
# Utility tools to put together the flowsheet and calculate the degrees of freedom
from idaes.models.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.solvers import get_solver
import idaes.core.util.scaling as iscale

# Import idaes logger to set output levels
import idaes.logger as idaeslog

## Importing required thermo and reaction packages

Finally, we import the thermophysical (`ideal_VLE.py` and `BTXParameterBlock`) packages and reaction package (`reaction.py`) for the HDA process. We have created custom thermophysical packages that assume ideal gas behavior with support for VLE. The reaction package consists of the stochiometric coefficients for the reaction, heat of reaction, and kinetic information (Arrhenius constant and activation energy). 

In [7]:
from idaes_examples.mod.hda import hda_reaction as reaction_props
from idaes.models.properties.activity_coeff_models.BTX_activity_coeff_VLE import (
    BTXParameterBlock,
)

from idaes_examples.mod.hda.hda_ideal_VLE import HDAParameterBlock

## Constructing the Flowsheet

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us create a ConcreteModel and add the flowsheet block to it. 

In [8]:
# Create a Pyomo Concrete Model to contain the problem
m = ConcreteModel()

# Add a steady state flowsheet block to the model
m.fs = FlowsheetBlock(dynamic=False)

We will now add the thermophysical and reaction packages to the flowsheet.

In [9]:
# Property package for benzene, toluene, hydrogen, methane mixture
m.fs.BTHM_params = HDAParameterBlock()

# Property package for the benzene-toluene mixture
m.fs.BT_params = BTXParameterBlock(
    valid_phase=("Liq", "Vap"), activity_coeff_model="Ideal"
)

# Reaction package for the HDA reaction
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
    property_package=m.fs.BTHM_params
)

## Adding Unit Models

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding the Mixer (assigned a name M101) and a Heater (assigned a name H101). Note that, all unit models need to be given a property package argument. In addition, the Mixer unit model needs a `list` consisting of the inlets (toluene feed, hydrogen feed and vapor recycle streams in this case). 

In [10]:
# Adding the mixer M101 to the flowsheet
m.fs.M101 = Mixer(
    property_package=m.fs.BTHM_params,
    inlet_list=["toluene_feed", "hydrogen_feed", "vapor_recycle"],
)

# Adding the heater H101 to the flowsheet
m.fs.H101 = Heater(property_package=m.fs.BTHM_params, has_phase_equilibrium=True)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now add the CSTR (assign the name R101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.BTHM_params</li>
         <li>"reaction_package": m.fs.reaction_params </li>
         <li>"has_heat_of_reaction": True </li>
         <li>"has_heat_transfer": True</li>
      </ul>
</div>

In [11]:
# Todo: Add reactor with the specifications above
m.fs.R101 = CSTR(
    property_package=m.fs.BTHM_params,
    reaction_package=m.fs.reaction_params,
    has_heat_of_reaction=True,
    has_heat_transfer=True,
)

Let us now add the Flash (assign the name F101), Splitter (assign the name S101) and PressureChanger (assign the name C101)

In [12]:
# Adding the flash tank F101 to the flowsheet
m.fs.F101 = Flash(
    property_package=m.fs.BTHM_params, has_heat_transfer=True, has_pressure_change=True
)

# Adding the splitter S101 to the flowsheet
m.fs.S101 = Splitter(
    property_package=m.fs.BTHM_params, outlet_list=["purge", "recycle"]
)

# Adding the compressor C101 to the flowsheet
m.fs.C101 = PressureChanger(
    property_package=m.fs.BTHM_params,
    compressor=True,
    thermodynamic_assumption=ThermodynamicAssumption.isothermal,
)

## Remark

Currently, the `SequentialDecomposition()` tool, which we will later be using to initialize the flowsheet, does not support the distillation column model. Thus, we will first simulate the flowsheet without the distillation column. After it converges, we will then add the distillation column, initialize it, and simulate the entire flowsheet.

As mentioned above, we use the `m.fs.BTHM_params` package, which contains all the four species, for the reactor loop, and the simpler `m.fs.BT_params` for unit operations following the flash (i.e., heater H102 and the distillation column D101). We define a `Translator` block to link the source property package and the package it is to be translated to in the following manner:

In [13]:
# Add translator block to convert between property packages
m.fs.translator = Translator(
    inlet_property_package=m.fs.BTHM_params, outlet_property_package=m.fs.BT_params
)

### Translator block constraints

The `Translator` block needs to know how to translate between the two property packages. This must be custom coded for each application because of the generality of the IDAES framework.

For this process, five constraints are required based on the state variables used in the outgoing process.

- Since we assumed that only benzene and toluene are present in the liquid phase, the total molar flowrate must be the sum of molar flowrates of benzene and toluene, respectively.
- Temperature of the inlet and outlet streams must be the same.
- Pressure of the inlet and outgoing streams must be the same
- The mole fraction of benzene in the outgoing stream is the ratio of the molar flowrate of liquid benzene in the inlet to the sum of molar flowrates of liquid benzene and toluene in the inlet.
- The mole fraction of toluene in the outgoing stream is the ratio of the molar flowrate of liquid toluene in the inlet to the sum of molar flowrates of liquid benzene and toluene in the inlet.

In [14]:
# Add constraint: Total flow = benzene flow + toluene flow (molar)
m.fs.translator.eq_total_flow = Constraint(
    expr=m.fs.translator.outlet.flow_mol[0]
    == m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
    + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
)

# Add constraint: Outlet temperature = Inlet temperature
m.fs.translator.eq_temperature = Constraint(
    expr=m.fs.translator.outlet.temperature[0] == m.fs.translator.inlet.temperature[0]
)

In the above, note that the variable flow_mol_phase_comp has the index - [time, phase, component]. As this is a steady-state flowsheet, the time index by default is 0. The valid phases are ["Liq", "Vap"]. Similarly the valid component list is ["benzene", "toluene", "hydrogen", "methane"].

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Add the constraint to ensure that the outlet pressure is the same as the inlet pressure
</div>

In [15]:
# Todo: Add constraint: Outlet pressure = Inlet pressure
m.fs.translator.eq_pressure = Constraint(
    expr=m.fs.translator.outlet.pressure[0] == m.fs.translator.inlet.pressure[0]
)

In [16]:
# Remaining constraints on the translator block

# Add constraint: Benzene mole fraction definition
m.fs.translator.eq_mole_frac_benzene = Constraint(
    expr=m.fs.translator.outlet.mole_frac_comp[0, "benzene"]
    == m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
    / (
        m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
        + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
    )
)

# Add constraint: Toluene mole fraction definition
m.fs.translator.eq_mole_frac_toluene = Constraint(
    expr=m.fs.translator.outlet.mole_frac_comp[0, "toluene"]
    == m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
    / (
        m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
        + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
    )
)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Finally, let us add the Heater H102 in the same way as H101 but pass the m.fs.BT_params thermodynamic package. We will add the distillation column after converging the flowsheet.
</div>

In [17]:
# Todo: Add the Heater H102 to the flowsheet
m.fs.H102 = Heater(
    property_package=m.fs.BT_params,
    has_pressure_change=True,
    has_phase_equilibrium=True,
)

## Connecting Unit Models using Arcs

We have now added the initial set of unit models to the flowsheet. However, we have not yet specifed how the units are connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer (M101) to the inlet of the heater (H101). 

In [18]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)


![](HDA_flowsheet_distillation.png) 

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, connect the H101 outlet to the R101 inlet using the cell above as a guide. 
</div>



In [19]:
# Todo: Connect the H101 outlet to R101 inlet
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)

We will now be connecting the rest of the units as shown below. Notice how the outlet names are different for the flash tank as it has a vapor and a liquid outlet. 

In [20]:
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet, destination=m.fs.M101.vapor_recycle)
m.fs.s10a = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.translator.inlet)
m.fs.s10b = Arc(source=m.fs.translator.outlet, destination=m.fs.H102.inlet)

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [21]:
TransformationFactory("network.expand_arcs").apply_to(m)

### Appending additional constraints to the model

Now, we will see how we can add additional constraints to the model using `Constraint` from Pyomo.

Consider the reactor R101. By default, the conversion of a component is not calculated when we simulate the flowsheet. If we are interested either in specifying or constraining the conversion value, we can add the following constraint to calculate the conversion:
$$ \text{Conversion of toluene} = \frac{\text{molar flow of toluene in the inlet} - \text{molar flow of toluene in the outlet}}{\text{molar flow of toluene in the inlet}} $$ 

We add the constraint to the model as shown below.

In [22]:
# Define the conversion variables using 'Var'
m.fs.R101.conversion = Var(initialize=0.75, bounds=(0, 1))

# Append the constraint to the model
m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion * m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"]
    == (
        m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"]
        - m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "toluene"]
    )
)

## Fixing feed conditions and Initializing the flowsheet

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. 

In [23]:
print(degrees_of_freedom(m))

29


We will now be fixing the toluene feed stream to the conditions shown in the flowsheet above. Please note that though this is a pure toluene feed, the remaining components are still assigned a very small non-zero value to help with convergence and initializing. 

In [24]:
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(0.30)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.temperature.fix(303.2)
m.fs.M101.toluene_feed.pressure.fix(350000)

Similarly, let us fix the hydrogen feed to the following conditions in the next cell:
      <ul>
         <li>F<sub>H2</sub> = 0.30 mol/s</li>
         <li>F<sub>CH4</sub> = 0.02 mol/s</li>
         <li>Remaining components = 1e-5 mol/s</li>
         <li>T = 303.2 K</li>
         <li>P = 350000 Pa</li>
      </ul>

In [25]:
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(0.30)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(0.02)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.hydrogen_feed.temperature.fix(303.2)
m.fs.M101.hydrogen_feed.pressure.fix(350000)

### Fixing unit model specifications

Now that we have fixed our inlet feed conditions, we will now be fixing the operating conditions for the unit models in the flowsheet. Let us set the H101 outlet temperature to 600 K. 

In [26]:
# Fix the temperature of the outlet from the heater H101
m.fs.H101.outlet.temperature.fix(600)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Set the conditions for the reactor R101 to the following conditions:
      <ul>
         <li>`conversion` = 0.75 </li>
         <li>`heat_duty` = 0</li>
      </ul>

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [27]:
# Todo: Fix the 'conversion' of the reactor R101
m.fs.R101.conversion.fix(0.75)

# Todo: Fix the 'heat_duty' of the reactor R101
m.fs.R101.heat_duty.fix(0)

The Flash conditions for F101 can be set as follows. 

In [28]:
# Fix the temperature of the vapor outlet from F101
m.fs.F101.vap_outlet.temperature.fix(325.0)

# Fix the pressure drop in the flash F101
m.fs.F101.deltaP.fix(0)

Let us fix the split fraction of the purge stream from the splitter S101 and the outlet pressure from the compressor C101

In [29]:
# Fix the split fraction of the 'purge' stream from S101
m.fs.S101.split_fraction[0, "purge"].fix(0.2)

# Fix the pressure of the outlet from the compressor C101
m.fs.C101.outlet.pressure.fix(350000)

Finally, let us fix the temperature of the outlet from H102 and the pressure drop in H102 as the following

In [30]:
# Fix the temperature of the outlet from the heater H102
m.fs.H102.outlet.temperature.fix(375)

# Fix the pressure drop in the heater H102
m.fs.H102.deltaP.fix(-200000)

To avoid convergence issues associated with poorly scaled variables and/or constraints, we scale the variables and constraints corresponding to the heaters H101 and H102, flash F101 and the reactor R101. Scaling factors for the flow rates, temperature, pressure, etc. have been defined in the property package: `ideal_VLE.py` file. Here, we set scaling factors only for the heat duty of the heater, the reaction extent, heat duty and volume of the reactor.

In [31]:
# Set scaling factors for heat duty, reaction extent and volume
iscale.set_scaling_factor(m.fs.H101.control_volume.heat, 1e-2)
iscale.set_scaling_factor(m.fs.R101.control_volume.heat, 1e-2)
iscale.set_scaling_factor(m.fs.R101.control_volume.rate_reaction_extent, 1)
iscale.set_scaling_factor(m.fs.R101.control_volume.volume, 1)
iscale.set_scaling_factor(m.fs.F101.control_volume.heat, 1e-2)
iscale.set_scaling_factor(m.fs.H102.control_volume.heat, 1e-2)

# Set the scaling factors for the remaining variables and all constraints
iscale.calculate_scaling_factors(m.fs.H101)
iscale.calculate_scaling_factors(m.fs.R101)
iscale.calculate_scaling_factors(m.fs.F101)
iscale.calculate_scaling_factors(m.fs.H102)

2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].flow_mol_phase


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].enth_mol_phase_comp[Liq,benzene]


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].enth_mol_phase_comp[Liq,toluene]


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].enth_mol_phase_comp[Vap,benzene]


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].enth_mol_phase_comp[Vap,toluene]


...

2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].material_flow_terms[Vap,benzene]


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].material_flow_terms[Liq,toluene]


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].material_flow_terms[Vap,toluene]


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].enthalpy_flow_terms[Liq], enthalpy_flow_terms


2023-11-02 10:27:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.H102.control_volume.properties_in[0.0].enthalpy_flow_terms[Vap], enthalpy_flow_terms


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now defined all the feed conditions and the inputs required for the unit models. The system should now have 0 degrees of freedom i.e. should be a square problem. Please check that the degrees of freedom is 0. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [32]:
# Todo: Check the degrees of freedom
print(degrees_of_freedom(m))

0


### Initialization

This subsection will demonstrate how to use the built-in sequential decomposition tool to initialize our flowsheet.

Let us first create an object for the `SequentialDecomposition` and specify our options for this. 

In [33]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 3

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

Which is the tear stream? Display tear set and order

In [34]:
for o in heuristic_tear_set:
    print(o.name)

fs.s03


What sequence did the SD tool determine to solve this flowsheet with the least number of tears? 

In [35]:
for o in order:
    print(o[0].name)

fs.H101
fs.R101
fs.F101
fs.S101
fs.C101
fs.M101


The SequentialDecomposition tool has determined that the tear stream is the mixer outlet (s03 in the Figure above). We will need to provide a reasonable guess for this.

For the initial guess, we assume that the flowrate of the recycle stream (s09) is zero. Consequently, the flow rate of the stream s03 is simply the sum of the flowrates of the toluene feed and hydrogen feed streams. Further, since the temperature and the pressure of both the toluene and hydrogen feed streams are the same, we specify their values as the initial guess for the temperature and pressure of the stream s03.

In [36]:
tear_guesses = {
    "flow_mol_phase_comp": {
        (0, "Vap", "benzene"): 1e-5,
        (0, "Vap", "toluene"): 1e-5,
        (0, "Vap", "hydrogen"): 0.30,
        (0, "Vap", "methane"): 0.02,
        (0, "Liq", "benzene"): 1e-5,
        (0, "Liq", "toluene"): 0.30,
        (0, "Liq", "hydrogen"): 1e-5,
        (0, "Liq", "methane"): 1e-5,
    },
    "temperature": {0: 303},
    "pressure": {0: 350000},
}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

Next, we need to tell the tool how to initialize a particular unit. We will be writing a python function which takes in a "unit" and calls the initialize method on that unit.

In [37]:
def function(unit):
    unit.initialize(outlvl=idaeslog.INFO)

We are now ready to initialize our flowsheet in a sequential mode. Note that we specifically set the iteration limit to be 3 as we are trying to use this tool only to get a good set of initial values such that IPOPT can then take over and solve this flowsheet for us. 

In [38]:
seq.run(m, function)

2023-11-02 10:27:19 [INFO] idaes.init.fs.H101.control_volume: Initialization Complete


2023-11-02 10:27:19 [INFO] idaes.init.fs.H101: Initialization Complete: optimal - Optimal Solution Found


2023-11-02 10:27:19 [INFO] idaes.init.fs.R101.control_volume: Initialization Complete


2023-11-02 10:27:19 [INFO] idaes.init.fs.R101: Initialization Complete: optimal - Optimal Solution Found


2023-11-02 10:27:19 [INFO] idaes.init.fs.F101.control_volume: Initialization Complete


...

2023-11-02 10:27:25 [INFO] idaes.init.fs.H102.control_volume.properties_out: Initialization Step 5 optimal - Optimal Solution Found.


2023-11-02 10:27:25 [INFO] idaes.init.fs.H102.control_volume.properties_out: State Released.


2023-11-02 10:27:25 [INFO] idaes.init.fs.H102.control_volume: Initialization Complete


2023-11-02 10:27:25 [INFO] idaes.init.fs.H102.control_volume.properties_in: State Released.


2023-11-02 10:27:25 [INFO] idaes.init.fs.H102: Initialization Complete: optimal - Optimal Solution Found


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now initialized the flowsheet. Let us run the flowsheet in a simulation mode to look at the results. 
</div>

In [39]:
# Create the solver object
solver = get_solver()

# Solve the model
results = solver.solve(m, tee=True)

'fs.H102.control_volume.properties_out[0.0].scaling_factor' that contains 1
component keys that are not exported as part of the NL file.  Skipping.


that contains 1 component keys that are not exported as part of the NL file.
Skipping.


'fs.F101.control_volume.properties_out[0.0].scaling_factor' that contains 26
component keys that are not exported as part of the NL file.  Skipping.


'fs.F101.control_volume.properties_in[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


that contains 1 component keys that are not exported as part of the NL file.
Skipping.


...

that contains 2 component keys that are not exported as part of the NL file.
Skipping.


'fs.H101.control_volume.properties_out[0.0].scaling_factor' that contains 26
component keys that are not exported as part of the NL file.  Skipping.


'fs.H101.control_volume.properties_in[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://www.hsl.rl.ac.

### Add distillation column 

As mentioned earlier, the `SequentialDecomposition` tool currently does not support the distillation column model. Thus, we have not included the distillation column in the flowsheet. Now that we have a converged flowsheet, we will add the distillation column and simulate the entire flowsheet. 

In the following, we will
- Add the distillation column 
- Connect it to the heater 
- Add the necessary equality constraints
- Propogate the state variable information from the outlet of the heater to the inlet of the distillation column 
- Fix the degrees of freedom of the distillation block (reflux ratio, boilup ratio, and condenser pressure)
- Scale the control volume heat variables to help convergence
- Initialize the distillation block.



In [40]:
# Add distillation column to the flowsheet
m.fs.D101 = TrayColumn(
    number_of_trays=10,
    feed_tray_location=5,
    condenser_type=CondenserType.totalCondenser,
    condenser_temperature_spec=TemperatureSpec.atBubblePoint,
    property_package=m.fs.BT_params,
)

# Connect the outlet from the heater H102 to the distillation column
m.fs.s11 = Arc(source=m.fs.H102.outlet, destination=m.fs.D101.feed)

# Add the necessary equality constraints
TransformationFactory("network.expand_arcs").apply_to(m)

# Propagate the state
propagate_state(m.fs.s11)

# Fix the reflux ratio, boilup ratio, and the condenser pressure
m.fs.D101.condenser.reflux_ratio.fix(0.5)
m.fs.D101.reboiler.boilup_ratio.fix(0.5)
m.fs.D101.condenser.condenser_pressure.fix(150000)

# set scaling factors
# Set scaling factors for heat duty
iscale.set_scaling_factor(m.fs.D101.condenser.control_volume.heat, 1e-2)
iscale.set_scaling_factor(m.fs.D101.reboiler.control_volume.heat, 1e-2)

# Set the scaling factors for the remaining variables and all constraints
iscale.calculate_scaling_factors(m.fs.D101)

# Initialize the distillation column
m.fs.D101.initialize(outlvl=idaeslog.INFO)

2023-11-02 10:27:25 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.D101.rectification_section[1].properties_in_liq[0.0].flow_mol_phase


2023-11-02 10:27:25 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.D101.rectification_section[1].properties_in_liq[0.0].eq_comp[benzene]


2023-11-02 10:27:25 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.D101.rectification_section[1].properties_in_liq[0.0].eq_comp[toluene]


2023-11-02 10:27:25 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.D101.rectification_section[1].properties_in_liq[0.0].eq_phase_equilibrium[benzene]


2023-11-02 10:27:25 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.D101.rectification_section[1].properties_in_liq[0.0].eq_phase_equilibrium[toluene]


...

2023-11-02 10:27:33 [INFO] idaes.init.fs.D101.rectification_section[1].properties_in_liq: State Released.


2023-11-02 10:27:33 [INFO] idaes.init.fs.D101: Column section + Condenser initialization status optimal - Optimal Solution Found.


2023-11-02 10:27:33 [INFO] idaes.init.fs.D101.stripping_section[10].properties_in_vap: State Released.


2023-11-02 10:27:33 [INFO] idaes.init.fs.D101: Column section + Condenser + Reboiler initialization status optimal - Optimal Solution Found.


2023-11-02 10:27:33 [INFO] idaes.init.fs.D101.feed_tray.properties_in_feed: State Released.


## Adding expressions to compute capital and operating costs

In this section, we will add a few Expressions that allow us to evaluate the performance. Expressions provide a convenient way of calculating certain values that are a function of the variables defined in the model. For more details on Expressions, please refer to: https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html

In [41]:
# Expression to compute the total cooling cost
m.fs.cooling_cost = Expression(
    expr=0.25e-7 * (-m.fs.F101.heat_duty[0])
    + 0.2e-7 * (-m.fs.D101.condenser.heat_duty[0])
)

# Expression to compute the total heating cost
m.fs.heating_cost = Expression(
    expr=2.2e-7 * m.fs.H101.heat_duty[0]
    + 1.2e-7 * m.fs.H102.heat_duty[0]
    + 1.9e-7 * m.fs.D101.reboiler.heat_duty[0]
)

# Expression to compute the total operating cost
m.fs.operating_cost = Expression(
    expr=(3600 * 24 * 365 * (m.fs.heating_cost + m.fs.cooling_cost))
)

# Expression to compute the total capital cost
m.fs.capital_cost = Expression(expr=1e5 * m.fs.R101.volume[0])

### Solve the entire flowsheet

In [42]:
solver.solve(m, tee=True)

'fs.D101.condenser.control_volume.properties_out[0.0].scaling_factor' that
contains 1 component keys that are not exported as part of the NL file.
Skipping.


'fs.H102.control_volume.properties_out[0.0].scaling_factor' that contains 1
component keys that are not exported as part of the NL file.  Skipping.


that contains 1 component keys that are not exported as part of the NL file.
Skipping.


'fs.F101.control_volume.properties_out[0.0].scaling_factor' that contains 26
component keys that are not exported as part of the NL file.  Skipping.


'fs.F101.control_volume.properties_in[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


...

that contains 2 component keys that are not exported as part of the NL file.
Skipping.


'fs.H101.control_volume.properties_out[0.0].scaling_factor' that contains 26
component keys that are not exported as part of the NL file.  Skipping.


'fs.H101.control_volume.properties_in[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes fo

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1169, 'Number of variables': 1169, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.08404064178466797}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Analyze the Results of the Square Problem

How much is the total cost (operating cost + capital cost), operating cost, capital cost, benzene purity in the distillate from the distilation column, and conversion of toluene in the reactor?

In [43]:
print("total cost = $", value(m.fs.capital_cost) + value(m.fs.operating_cost))
print("operating cost = $", value(m.fs.operating_cost))
print("capital cost = $", value(m.fs.capital_cost))
print()
print(
    "Distillate flowrate = ",
    value(m.fs.D101.condenser.distillate.flow_mol[0]()),
    "mol/s",
)
print(
    "Benzene purity = ",
    100 * value(m.fs.D101.condenser.distillate.mole_frac_comp[0, "benzene"]),
    "%",
)
print("Residue flowrate = ", value(m.fs.D101.reboiler.bottoms.flow_mol[0]()), "mol/s")
print(
    "Toluene purity = ",
    100 * value(m.fs.D101.reboiler.bottoms.mole_frac_comp[0, "toluene"]),
    "%",
)
print()
print("Conversion = ", 100 * value(m.fs.R101.conversion), "%")
print()
print(
    "Overhead benzene loss in F101 = ",
    100
    * value(m.fs.F101.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"])
    / value(m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"]),
    "%",
)

total cost = $ 442301.4707525308
operating cost = $ 427596.7305680824
capital cost = $ 14704.74018444835

Distillate flowrate =  0.16196898920642744 mol/s
Benzene purity =  89.49161665800828 %
Residue flowrate =  0.1051500712068829 mol/s
Toluene purity =  43.32260291055269 %

Conversion =  75.0 %

Overhead benzene loss in F101 =  42.16193848360187 %


Get the state of the streams entering and leaving the reactor R101

In [44]:
m.fs.R101.report()


Unit : fs.R101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value   : Units      : Fixed : Bounds
    Heat Duty :  0.0000 :       watt :  True : (None, None)
       Volume : 0.14705 : meter ** 3 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                Units         Inlet     Outlet  
    flow_mol_phase_comp ('Liq', 'benzene')   mole / second 1.2993e-07 1.2993e-07
    flow_mol_phase_comp ('Liq', 'toluene')   mole / second 8.4147e-07 8.4147e-07
    flow_mol_phase_comp ('Liq', 'methane')   mole / second 1.0000e-12 1.0000e-12
    flow_mol_phase_comp ('Liq', 'hydrogen')  mole / second 1.0000e-12 1.0000e-12
    flow_mol_phase_comp ('Vap', 'benzene')   mole / second    0.11936    0.35374
    flow_mol_phase_comp ('V

Get the state of the streams entering and leaving the reactor R101

In [45]:
m.fs.F101.report()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value   : Units  : Fixed : Bounds
          Heat Duty : -70343. :   watt : False : (None, None)
    Pressure Change :  0.0000 : pascal :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                Units         Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')   mole / second 1.2993e-07   1.0000e-08       0.20460  
    flow_mol_phase_comp ('Liq', 'toluene')   mole / second 8.4147e-07   1.0000e-08      0.062520  
    flow_mol_phase_comp ('Liq', 'methane')   mole / second 1.0000e-12   1.0000e-08    2.6712e-07  
    flow_mol_phase_comp ('Liq', 'hydrogen')  mole / second 1.0000e-12   1.0000e-08    2.6712e-07  
    flow_mol

Next, let's look at how much benzene we are losing with the light gases out of F101. IDAES has tools for creating stream tables based on the `Arcs` and/or `Ports` in a flowsheet. Let us create and print a simple stream table showing the stream leaving the reactor and the vapor stream from F101.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
How much benzene are we losing in the F101 vapor outlet stream?
</div>

In [46]:
from idaes.core.util.tables import (
    create_stream_table_dataframe,
    stream_table_dataframe_to_string,
)

st = create_stream_table_dataframe({"Reactor": m.fs.s05, "Light Gases": m.fs.s06})
print(stream_table_dataframe_to_string(st))

                                            Units        Reactor   Light Gases
flow_mol_phase_comp ('Liq', 'benzene')   mole / second 1.2993e-07  1.0000e-08 
flow_mol_phase_comp ('Liq', 'toluene')   mole / second 8.4147e-07  1.0000e-08 
flow_mol_phase_comp ('Liq', 'methane')   mole / second 1.0000e-12  1.0000e-08 
flow_mol_phase_comp ('Liq', 'hydrogen')  mole / second 1.0000e-12  1.0000e-08 
flow_mol_phase_comp ('Vap', 'benzene')   mole / second    0.35374     0.14915 
flow_mol_phase_comp ('Vap', 'toluene')   mole / second   0.078129    0.015610 
flow_mol_phase_comp ('Vap', 'methane')   mole / second     1.2721      1.2721 
flow_mol_phase_comp ('Vap', 'hydrogen')  mole / second    0.32821     0.32821 
temperature                                     kelvin     771.85      325.00 
pressure                                        pascal 3.5000e+05  3.5000e+05 


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
You can querry additional variables here if you like. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

# Optimization


We saw from the results above that the total operating cost for the base case was $442,297 per year. We are producing 0.162 mol/s of benzene at a purity of 89.5%. However, we are losing around 43.3% of benzene in F101 vapor outlet stream. 

Let us try to minimize this cost such that:
- we are producing at least 0.18 mol/s of benzene as distillate i.e. our product stream
- purity of benzene i.e. the mole fraction of benzene in the distillate is at least 99%
- restricting the benzene loss in F101 vapor outlet to less than 20%

For this problem, our decision variables are as follows:
- H101 outlet temperature
- R101 outlet temperature
- F101 outlet temperature
- H102 outlet temperature
- Condenser pressure
- reflux ratio
- boilup ratio


Let us declare our objective function for this problem. 

In [47]:
m.fs.objective = Objective(expr=m.fs.operating_cost + m.fs.capital_cost)

Now, we need to unfix the decision variables as we had solved a square problem (degrees of freedom = 0) until now. 

In [48]:
m.fs.H101.outlet.temperature.unfix()
m.fs.R101.conversion.unfix()
m.fs.F101.vap_outlet.temperature.unfix()
m.fs.D101.condenser.condenser_pressure.unfix()
m.fs.D101.condenser.reflux_ratio.unfix()
m.fs.D101.reboiler.boilup_ratio.unfix()

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now unfix the remaining variable: the temperature of the outlet from H102

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [49]:
# Todo: Unfix the temperature of the outlet from H102
m.fs.H102.outlet.temperature.unfix()

Next, we need to set bounds on these decision variables to values shown below:

 - H101 outlet temperature [500, 600] K
 - R101 outlet temperature [600, 900] K
 - F101 outlet temperature [298, 450] K
 - H102 outlet temperature [350, 400] K
 - D101 condenser pressure [101325, 150000] Pa
 - D101 reflux ratio [0.1, 5]
 - D101 boilup ratio [0.1, 5]

In [50]:
# Set bounds on the temperature of the outlet from H101
m.fs.H101.outlet.temperature[0].setlb(500)
m.fs.H101.outlet.temperature[0].setub(600)

# Set bounds on the temperature of the outlet from R101
m.fs.R101.outlet.temperature[0].setlb(600)
m.fs.R101.outlet.temperature[0].setub(900)

# Set bounds on the volume of the reactor R101
m.fs.R101.volume[0].setlb(0)

# Set bounds on the temperature of the vapor outlet from F101
m.fs.F101.vap_outlet.temperature[0].setlb(298)
m.fs.F101.vap_outlet.temperature[0].setub(450.0)

# Set bounds on the temperature of the outlet from H102
m.fs.H102.outlet.temperature[0].setlb(350)
m.fs.H102.outlet.temperature[0].setub(400)

# Set bounds on the pressure inside the condenser
m.fs.D101.condenser.condenser_pressure.setlb(101325)
m.fs.D101.condenser.condenser_pressure.setub(150000)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, set the bounds for the D101 reflux ratio and boilup ratio.

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [51]:
# Todo: Set bounds on the reflux ratio
m.fs.D101.condenser.reflux_ratio.setlb(0.1)
m.fs.D101.condenser.reflux_ratio.setub(5)

# Todo: Set bounds on the boilup ratio
m.fs.D101.reboiler.boilup_ratio.setlb(0.1)
m.fs.D101.reboiler.boilup_ratio.setub(5)

Now, the only things left to define are our constraints on overhead loss in F101, distillate flowrate and its purity. Let us first look at defining a constraint for the overhead loss in F101 where we are restricting the benzene leaving the vapor stream to less than 20 % of the benzene available in the reactor outlet. 

In [52]:
# Ensure that the overhead loss of benzene from F101 <= 20%
m.fs.overhead_loss = Constraint(
    expr=m.fs.F101.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
    <= 0.20 * m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, add the constraint such that we are producing at least 0.18 mol/s of benzene in the product stream which is the distillate of D101. Let us name this constraint as m.fs.product_flow. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [53]:
# Todo: Add minimum product flow constraint
m.fs.product_flow = Constraint(expr=m.fs.D101.condenser.distillate.flow_mol[0] >= 0.18)

Let us add the final constraint on product purity or the mole fraction of benzene in the distillate such that it is at least greater than 99%. 

In [54]:
m.fs.product_purity = Constraint(
    expr=m.fs.D101.condenser.distillate.mole_frac_comp[0, "benzene"] >= 0.99
)


We have now defined the optimization problem and we are now ready to solve this problem. 




In [55]:
results = solver.solve(m, tee=True)

that contains 1 component keys that are not exported as part of the NL file.
Skipping.


'fs.F101.control_volume.properties_out[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


'fs.F101.control_volume.properties_in[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


that contains 1 component keys that are not exported as part of the NL file.
Skipping.


'fs.R101.control_volume.properties_out[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


...

'fs.H101.control_volume.properties_in[0.0].scaling_factor' that contains 25
component keys that are not exported as part of the NL file.  Skipping.


Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://www.hsl.rl.ac.

   4  4.2249096e+05 1.45e+05 1.43e+03  -1.0 7.01e+06    -  5.14e-01 2.03e-01h  1
   5  4.2194364e+05 8.17e+04 1.70e+04  -1.0 6.06e+06    -  5.97e-01 4.28e-01h  1
   6  4.2602765e+05 4.55e+04 1.10e+06  -1.0 4.32e+06    -  9.26e-01 5.07e-01h  1
   7  4.3776643e+05 2.03e+04 6.44e+09  -1.0 2.42e+06    -  9.90e-01 9.47e-01h  1
   8  4.3846260e+05 1.92e+04 6.05e+09  -1.0 4.42e+05    -  5.40e-01 5.74e-02h  1
   9  4.4529853e+05 4.05e+04 4.66e+10  -1.0 2.47e+05    -  9.96e-01 9.90e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  4.4906283e+05 9.76e+03 1.10e+10  -1.0 1.12e+03  -4.0 1.26e-01 7.45e-01h  1
  11  4.5079086e+05 1.19e+03 1.54e+09  -1.0 5.63e+02  -4.5 3.77e-01 1.00e+00h  1
  12  4.5024224e+05 2.66e+00 3.67e+06  -1.0 6.61e+01  -5.0 1.00e+00 1.00e+00f  1
  13  4.4946170e+05 5.64e-01 9.29e+05  -1.0 1.81e+02  -5.4 1.00e+00 7.88e-01f  1
  14  4.4916780e+05 8.48e+00 1.62e+05  -1.0 2.83e+02  -5.9 1.00e+00 1.00e+00f  1


  15  4.4899127e+05 4.83e+00 9.06e+04  -1.0 1.01e+02  -6.4 1.00e+00 4.40e-01f  2
  16  4.4886718e+05 7.00e-01 4.61e+02  -1.0 2.35e+02  -6.9 1.00e+00 1.00e+00f  1
  17  4.4800159e+05 1.39e+02 4.52e+06  -3.8 1.17e+03  -7.3 9.79e-01 9.37e-01f  1
  18  4.4672196e+05 9.59e+02 1.22e+06  -3.8 4.55e+03  -7.8 1.00e+00 9.43e-01f  1
  19  4.4401667e+05 7.75e+03 1.55e+05  -3.8 1.08e+04  -8.3 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  4.4185035e+05 1.91e+04 1.36e+04  -3.8 1.33e+04  -8.8 1.00e+00 1.00e+00h  1
  21  4.4241001e+05 3.52e+03 5.96e+03  -3.8 2.94e+03  -9.2 1.00e+00 1.00e+00h  1
  22  4.4185237e+05 7.82e+00 2.91e+02  -3.8 7.13e+03  -9.7 2.39e-01 1.00e+00h  1
  23  4.4124091e+05 1.53e+01 3.11e+02  -3.8 4.82e+04 -10.2 8.59e-01 1.36e-01f  1
  24  4.4137379e+05 1.80e+00 2.91e+02  -3.8 1.41e+04    -  1.95e-01 1.00e+00h  1
  25  4.3862833e+05 1.70e+03 9.48e+04  -3.8 1.57e+07    -  1.29e-03 9.10e-02f  1
  26  4.3883308e+05 1.49e+03

## Optimization Results

Display the results and product specifications

In [56]:
print("total cost = $", value(m.fs.capital_cost) + value(m.fs.operating_cost))
print("operating cost = $", value(m.fs.operating_cost))
print("capital cost = $", value(m.fs.capital_cost))
print()
print(
    "Distillate flowrate = ",
    value(m.fs.D101.condenser.distillate.flow_mol[0]()),
    "mol/s",
)
print(
    "Benzene purity = ",
    100 * value(m.fs.D101.condenser.distillate.mole_frac_comp[0, "benzene"]),
    "%",
)
print("Residue flowrate = ", value(m.fs.D101.reboiler.bottoms.flow_mol[0]()), "mol/s")
print(
    "Toluene purity = ",
    100 * value(m.fs.D101.reboiler.bottoms.mole_frac_comp[0, "toluene"]),
    "%",
)
print()
print("Conversion = ", 100 * value(m.fs.R101.conversion), "%")
print()
print(
    "Overhead benzene loss in F101 = ",
    100
    * value(m.fs.F101.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"])
    / value(m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"]),
    "%",
)

total cost = $ 438839.8984262867
operating cost = $ 408883.53148308955
capital cost = $ 29956.366943197143

Distillate flowrate =  0.17999999002639888 mol/s
Benzene purity =  98.99999900049087 %
Residue flowrate =  0.10851616424263709 mol/s
Toluene purity =  15.676178086212413 %

Conversion =  93.38705916369456 %

Overhead benzene loss in F101 =  17.340617931156157 %


Display optimal values for the decision variables

In [57]:
print("Optimal Values")
print()

print("H101 outlet temperature = ", value(m.fs.H101.outlet.temperature[0]), "K")

print()
print("R101 outlet temperature = ", value(m.fs.R101.outlet.temperature[0]), "K")

print()
print("F101 outlet temperature = ", value(m.fs.F101.vap_outlet.temperature[0]), "K")

print()
print("H102 outlet temperature = ", value(m.fs.H102.outlet.temperature[0]), "K")

Optimal Values

H101 outlet temperature =  568.9232042951961 K

R101 outlet temperature =  790.3655425698863 K

F101 outlet temperature =  298.0 K

H102 outlet temperature =  368.7414339952852 K


### Key Takeaways

Observe that the optimization was able to reduce the yearly operating cost from \\$427,593 to \\$408,342 (~4.5%). However, the amortized capital cost more than doubled from \\$14,704 to \\$29,927 due to the need to increase the conversion in the reactor (from 75% to 93%) to meet the production and purity constraints. 

Further, observe that the product flow rate and product purity are at their minimum values (0.18 mol/s and 99%, respectively). This is expected as increasing recovery would require more energy and cost to purify the product.


Finally, observe that the operating temperature of the flash (F101) is almost at its lower bound. This helps in minimizing the amount of benzene in the vapor stream leaving the flash.